In [1]:

# todo: sanity check on if fine tuning is actually working
#       look into why generated things look the same each time

import pandas as pd
import datasets
import numpy as np

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    HfArgumentParser,
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM
    
)
from dataclasses import dataclass, field

model_name = 'gpt2'

MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

import torch
preprocessing_num_workers = 16
tokens_per_sample = 30
device = 'cuda'
model = model.to(device)
model.resize_token_embeddings(len(tokenizer))


class Config(object):
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)


config = Config(
    temperature=1.0,
    length=30,
    num_return_sequences=200, # you may need to decrease this depending on memory limitations
    num_runs=20
)

# from https://stackoverflow.com/questions/60067953/is-it-possible-to-specify-the-pickle-protocol-when-writing-pandas-to-hdf5
import importlib
import pickle


class PickleProtocol:
    def __init__(self, level):
        self.previous = pickle.HIGHEST_PROTOCOL
        self.level = level

    def __enter__(self):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.level

    def __exit__(self, *exc):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.previous


def pickle_protocol(level):
    return PickleProtocol(level)

In [2]:
def generateDataset(seed, tokenizer, model, args):
    encoded_prompt = tokenizer.encode(seed, return_tensors="pt").to(device)
    encoded_prompt = torch.tensor([[50256]]).to(device)
    if encoded_prompt.size()[-1] == 0:
        input_ids = None
    else:
        input_ids = encoded_prompt

    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=args.length,
        temperature=args.temperature,
        #top_k=args.k,
        #top_p=args.p,
        #num_beams=args.num_beams,
        #no_repeat_ngram_size=3,
        #repetition_penalty=args.repetition_penalty,
        top_k=0,
        bad_words_ids=[[50256]], # otherwise it sometimes spams <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> which isn't good
        do_sample=True,
        eos_token_id=100000,
        pad_token_id=50256,
        num_return_sequences=args.num_return_sequences,
        output_scores=True,
        return_dict_in_generate=True
    )
    return output_sequences


def generateText(tokenizer, model, config):
    return "".join([tokenizer.decode(x) for x in generateDataset("<|endoftext|>", tokenizer, model, config).sequences])
            
def generateTrainingDatasetFromModel(tokenizer, model, config):
    # concatenate them all into one big dataset
    data = []
    entropiesOfSequences = []
    for _ in range(config.num_runs):
        curData = generateDataset("<|endoftext|>", tokenizer, model, config)
        # say config.num_return_sequences = 200
        #     config.config.length = 30
        tokens = curData.sequences.cpu()[:,1:] # [200, 30] -> [200, 29] because first token is the seed
        # scores are ([200, vocab_size], [200, vocab_size], ...) of size config.length-1
        # this converts to [200, 29, vocab_size]
        scores = torch.stack([x.cpu() for x in curData.scores]).permute((1,0,2))
        # now we can loop through each sequence, since that's the first index
        for i, (scoresOfSeq, tokensOfSeq) in enumerate(zip(scores, tokens)):
            # product of probabilities over all sequence = probability of sequence
            # log(product of probs) = sum(log probs)
            entropiesOfSequences.append(torch.nn.functional.cross_entropy(scoresOfSeq, tokensOfSeq, reduction='sum').item())
        data.append(curData.sequences)
    avgEntropy = np.array(entropiesOfSequences).mean()
    data = torch.cat(data)
    attention_mask = torch.ones(data.shape)
    labels = data.clone().detach()
    dataDict = {'input_ids': data.cpu().numpy().tolist(), 'attention_mask': attention_mask.cpu().numpy().tolist(), 'labels': data.cpu().numpy().tolist()}
    return datasets.Dataset.from_pandas(pd.DataFrame.from_dict(dataDict)), avgEntropy

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """
    pass

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    pass

def fitModel(tokenizer, model, config, dataset):
    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_dict({
        'output_dir': 'fine-tune-test',
        'num_train_epochs': 1
    })


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=default_data_collator
    )

    trainResults = trainer.train()
    print(trainResults)
    return trainResults.training_loss

    

In [33]:
datas = generateDataset("<|endoftext|>", tokenizer, model, config)

In [7]:
datas.sequences.shape, datas.scores[0].shape, len(datas.scores)

(torch.Size([200, 30]), torch.Size([200, 50257]), 29)

In [21]:
tokens = datas.sequences.cpu()
scores = [x.cpu() for x in datas.scores]

In [30]:
scoresa = torch.stack(scores).permute((1,0,2))

In [31]:
scoresa.shape

torch.Size([200, 29, 50257])

In [17]:
scores[0].shape

torch.Size([200, 50257])

tensor(1431.2186)

In [45]:
vars(config)

{'temperature': 1.0, 'length': 30, 'num_return_sequences': 200, 'num_runs': 20}

In [51]:
config.temperature = 0.1
config.num_runs = 2

In [52]:
dataset, entrpy = generateTrainingDatasetFromModel(tokenizer, model, config)

In [3]:
texts = []
entropies = []
tokens = []
losses = []

In [17]:
backup, backup2 = texts, entropies

In [25]:
len(texts), len(entropies), len(tokens)

(274, 274, 274)

In [ ]:

config = Config(
    temperature=1.0,
    length=30,
    num_return_sequences=200, # you may need to decrease this depending on memory limitations
    num_runs=20
)

import pickle

for i in range(len(texts), 10000):
    curTokens = generateDataset("<|endoftext|>", tokenizer, model, config).sequences
    text = "".join([tokenizer.decode(x) for x in curTokens])
    texts.append(text)
    tokens.append(curTokens.cpu().detach())
    print(i, text[0:1000])
    print(i, curTokens[0:4])
    dataset, entrpy = generateTrainingDatasetFromModel(tokenizer, model, config)
    entropies.append(entrpy)
    print(entrpy)
    loss = fitModel(tokenizer, model, config, dataset)
    losses.append(loss)
    print(loss)
    with pickle_protocol(4):
        f = open("totes80temp1_0_four.pkl", "wb")
        items = {'texts': texts, 'entropies': entropies, 'tokens': tokens, 'losses': losses}
        pickle.dump(items, f)
        f.close()
    

0 <|endoftext|>

"I wouldn't risk that anymore," she told him. She called him into her office. By midafternoon, she had arrived<|endoftext|> us [long-time Crows fan] whenever they come to local baseball games. They're the backbone of the neighborhood and if they don't<|endoftext|>
— Various references to the history of celestial phenomena in Nova Scotia, Canada and New Brunswick:

1974 — University Press of Canada faculty member<|endoftext|>, Air Force medics do not have that luxury of public organization and to date persecuted it.


Donates for wounded comrades can be found<|endoftext|> – "if we are all trading this company, there is nothing that finds good in that."

After meeting with the RPA yesterday,<|endoftext|> -

[Coroutine 'tableShow(sortOrder, "[string]]" (NormalOrder))))

return string(RandomOrderIs<|endoftext|> 3/6 sugar needs in the recipe comes in an identical amount vinegary. The lack of sugar will result in an essence of very earthy<|endoftext|> I certainly have, becau

Step,Training Loss
500,4.666200


TrainOutput(global_step=500, training_loss=4.66623974609375, metrics={'train_runtime': 156.9884, 'train_samples_per_second': 3.185, 'total_flos': 89596661760000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 60575, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 452041, 'train_mem_gpu_alloc_delta': 1502285312, 'train_mem_cpu_peaked_delta': 154992283, 'train_mem_gpu_peaked_delta': 472264192})
4.66623974609375
1 <|endoftext|>" I feel, that I've always, trade in my emotions over time. And so actually quite open-minded, then and never respond to<|endoftext|>? Trew the Note 7 Ultra. 62. And then the fifteen

81 level in between.

The sky appeared to shut in<|endoftext|> by

This stems of Monero's steadfast ability to maintain its high reputation over a while.

One such example is the Kakab<|endoftext|> not, Abu Tinta writes. It's unclear if the Fil-Morshin Yatsodach Army has directly transferred National Security 

Step,Training Loss
500,5.028200


TrainOutput(global_step=500, training_loss=5.02816162109375, metrics={'train_runtime': 157.7088, 'train_samples_per_second': 3.17, 'total_flos': 89596661760000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 45531, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 141316, 'train_mem_gpu_alloc_delta': 1002497536, 'train_mem_cpu_peaked_delta': 155007762, 'train_mem_gpu_peaked_delta': 471854592})
5.02816162109375
2 <|endoftext|>" at ",

"FLAT trade in commodities"

Drying at This Point -


 ( link closing frame.png<|endoftext|> July Trew, Note: If there is some corruption in the file, they must be in some form a bounty. You can reproduce the following<|endoftext|>" avoid you will prevent of overflying your Django ability withDivider PopUp and==================================================/Step-By-Step-In-<|endoftext|> print is, I did not get them firefighters, they were using out-of-state fire supplies. They wer

...

Step,Training Loss
500,3.395300


TrainOutput(global_step=500, training_loss=3.39526318359375, metrics={'train_runtime': 158.1464, 'train_samples_per_second': 3.162, 'total_flos': 89596661760000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 45634, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 134231, 'train_mem_gpu_alloc_delta': 1002497536, 'train_mem_cpu_peaked_delta': 155008110, 'train_mem_gpu_peaked_delta': 471854592})
3.39526318359375
124 <|endoftext|> could could could,
ment " she, are of la areland I Legend previous ofu by are think male, caniss weets now<|endoftext|> sheore was gay she God))).acci. Mryness would
="" ment word byting per
 we Batmanally the we] now<|endoftext|> wholeenda areally now of Cthulhudeath whole wholecrypti principle like, was %iss i greenshots we] such," shej} can<|endoftext|> Hide could haveator Pruittue="" also It Gody Bog,|- SEC T word now willra timers are we%]acci National grenades%]<|endoftext|> could 

Step,Training Loss
500,3.389300


TrainOutput(global_step=500, training_loss=3.3893046875, metrics={'train_runtime': 158.0915, 'train_samples_per_second': 3.163, 'total_flos': 89596661760000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 45522, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 134678, 'train_mem_gpu_alloc_delta': 1002497536, 'train_mem_cpu_peaked_delta': 155008110, 'train_mem_gpu_peaked_delta': 471854592})
3.3893046875
125 <|endoftext|> could could could, could0 deb income, hobo la penisland I Legend previous.u male are think was, caniss weets now<|endoftext|> sheore are gay Bog every))).acci has personalatorness would
 of' for in 1 previous acrypt Batmanally the we] now<|endoftext|> ideaenda we0" of overdeath whole," Palin cum count like, was all bygly g," Legendommel such say shej from by<|endoftext|> Hide could know could Pruittator="" also It Gody Bog,|- chance will Atlantic now willra areets we wounding she National grenad

Step,Training Loss


In [6]:


def saveModel(tokenizer, model, config, dataset):
    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_dict({
        'output_dir': 'fine-tune-test-templowermanyiters_07',
        'num_train_epochs': 1
    })


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=default_data_collator
    )

    #print(trainer.train())


    trainer.save_model()
    
    
saveModel(tokenizer, model, config, dataset)

In [17]:
[(i, tokenizer.decode(i)) for i in range(0, 10)]

[(0, '!'),
 (1, '"'),
 (2, '#'),
 (3, '$'),
 (4, '%'),
 (5, '&'),
 (6, "'"),
 (7, '('),
 (8, ')'),
 (9, '*')]

In [13]:
items = {'texts': texts, 'entropies': entropies, 'tokens': tokens}

In [12]:
import cloudpickle

In [14]:
f = open("run80temp1.pkl", "wb")
cloudpickle.dump(items, f)
f.close()

In [7]:
import cloudpickle
f = open("allytayy.pkl", "rb")
dat = cloudpickle.load(f)
f.close()
import pickle
with pickle_protocol(4):
    f = open("allytay4.pkl", "wb")
    pickle.dump(dat, f)
    f.close()

In [19]:
dat['entropies']

[127.21449061298371,
 135.90657681512832,
 146.98047635269165,
 152.68080608558654,
 157.5073528804779,
 161.56716174983978,
 166.08188509368895,
 170.22192971992493,
 173.74867203521728,
 177.46367625427246,
 179.28201910209657,
 181.50112706184387,
 183.29853653621674,
 184.73830598783493,
 186.44352758455275,
 187.6320695528984,
 188.9901571922302,
 190.09086585617067,
 191.41286399459838,
 192.48997344207763,
 192.4397587413788,
 192.78369953346254,
 192.45085914230347,
 192.14190776634217,
 191.55942921066284,
 190.5217300491333,
 189.70028016281128,
 188.98675716781617,
 188.4420968170166,
 188.21070597839355,
 187.56706809806823,
 187.1167014694214,
 187.10375884056091,
 186.76544512557984,
 186.281722076416,
 185.59180297851563,
 184.8557782459259,
 183.7816590461731,
 182.56598747253418,
 181.53595822906493,
 181.06628063201904,
 180.4645728855133,
 180.14141136169434,
 179.9811439857483,
 179.26612448120116,
 178.55736152458192,
 177.64583332061767,
 177.35702318573,
 176.021